In [1]:
import pandas as pd
import csv
import re
import string
import nltk

from pythainlp.tokenize import Tokenizer
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import dict_trie
from pythainlp.tag import pos_tag

from nltk.corpus import words
from stop_words import get_stop_words

# Processing text function

In [2]:
# word correction for EN
#parser = GingerIt()

# download stopwords of TH and EN
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))

# clean the text
def clean_text(msg): 

    msg = str(msg).lower()
    msg = re.sub(r"\,", "", str(msg))
    msg = re.sub(r"[^a-z0-9ก-๙\.\%\:]", " ", str(msg))
    
    # Delete punctuation
   # str_punc = string.punctuation.replace(".","")
   # for c in str_punc:
    #    msg = re.sub(r'\{}'.format(c),'',str(msg))
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


# Load data

In [3]:
# input
# import data from csv
file_name = 'naiin_full_data'

FILE = './Data/{}.csv'.format(file_name)
df = pd.read_csv(FILE,index_col=0)

In [4]:
# Drop column that have any "Na"
data = df.dropna(axis=1, how='any')

In [5]:
data.head()

,Box set แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกแข็ง),แพ็กชุด แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกอ่อน),แฮร์รี่พอตเตอร์กับภาคีนกฟีนิกซ์ล.5 20ปี,แฮร์รี่พอตเตอร์กับเจ้าชายเลือดผสมล.6 20ป,แฮร์รี่พอตเตอร์กับห้องแห่งความลับ ล2 20ป,แฮร์รี่ พอตเตอร์กับศิลาอาถรรพ์ ล.1 20ปี,แฮร์รี่พอตเตอร์กับนักโทษแห่งอัซคาบันล.3,แฮร์รี่พอตเตอร์กับเครื่องรางยมทูตล.7 20ป,แฮร์รี่พอตเตอร์กับถ้วยอัคนี ล.4 20ปี,คนใจดำ,...,เย็นสบายชายนํ้า (ใหม่ล่าสุด).1,สิงคโปร์สัญจร (ใหม่ล่าสุด).1,โรมันสัญจร (ใหม่ล่าสุด).1,Kaew the Naughty(แก้วจอมซน/ภาษาอังกฤษ).1,ใต้เมฆที่เมฆใต้ (ใหม่ล่าสุด).1,ชุดพระมหาชนกและหนังสือทองแดง.1,เยือนถิ่นจีนโพ้นทะเล (ใหม่ล่าสุด).1,ข้ามฝั่งแห่งฝัน (ใหม่ล่าสุด).1,คืนถิ่นจีนใหญ่ (ใหม่ล่าสุด).1,บันทึกภาพฯเมื่อครั้งในหลวงทรงผนวช.1
0,ผู้เขียน : J.K. Rowling,ผู้เขียน : J.K. Rowling,ผู้เขียน : J.K. Rowling,ผู้เขียน : J.K. Rowling,ผู้เขียน : J.K. Rowling,ผู้เขียน : J.K. Rowling,ผู้เขียน : J.K. Rowling,ผู้เขียน : J.K. Rowling,ผู้เขียน : J.K. Rowling,ผู้เขียน : นาคาลัย,...,ผู้เขียน : สมเด็จพระกนิษฐาธิราชเจ้า กรมสมเด็จพ...,ผู้เขียน : สมเด็จพระกนิษฐาธิราชเจ้า กรมสมเด็จพ...,ผู้เขียน : สมเด็จพระกนิษฐาธิราชเจ้า กรมสมเด็จพ...,ผู้เขียน : สมเด็จพระกนิษฐาธิราชเจ้า กรมสมเด็จพ...,ผู้เขียน : สมเด็จพระกนิษฐาธิราชเจ้า กรมสมเด็จพ...,ผู้เขียน : พระบาทสมเด็จพระปรมินทรมหาภูมิพลอดุล...,ผู้เขียน : สมเด็จพระกนิษฐาธิราชเจ้า กรมสมเด็จพ...,ผู้เขียน : สมเด็จพระกนิษฐาธิราชเจ้า กรมสมเด็จพ...,ผู้เขียน : สมเด็จพระกนิษฐาธิราชเจ้า กรมสมเด็จพ...,ผู้เขียน : พินิจ จันทร
1,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 3...,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 3...,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 9...,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 9...,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 9...,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 9...,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 9...,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 9...,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 9...,สำนักพิมพ์ : ดีต่อใจ<end line>บาร์โค้ด : 97861...,...,สำนักพิมพ์ : มูลนิธิสมเด็จพระเทพฯ<end line>บาร...,สำนักพิมพ์ : มูลนิธิสมเด็จพระเทพฯ<end line>บาร...,สำนักพิมพ์ : มูลนิธิสมเด็จพระเทพฯ<end line>บาร...,สำนักพิมพ์ : นานมีบุ๊คส์<end line>บาร์โค้ด : 9...,สำนักพิมพ์ : มูลนิธิสมเด็จพระเทพฯ<end line>บาร...,สำนักพิมพ์ : วังสวนจิตรลดา<end line>บาร์โค้ด :...,สำนักพิมพ์ : มูลนิธิสมเด็จพระเทพฯ<end line>บาร...,สำนักพิมพ์ : มูลนิธิสมเด็จพระเทพฯ<end line>บาร...,สำนักพิมพ์ : มูลนิธิสมเด็จพระเทพฯ<end line>บาร...,สำนักพิมพ์ : ปัญญาชน<end line>บาร์โค้ด : 97861...


In [6]:
def split_line(data):
    # Split each line by '<end line>'
    data_split = {}
    for k,v in data.iteritems():
        tmp = [t.split('<end line>') for t in v]
    
        # Split attribute and value by ':'
        for l in tmp:
            attr_val = [clean_text(s).split(':') for s in l]
        data_split[k] = attr_val
    return data_split

In [7]:
%%time
data_split = split_line(data)

Wall time: 260 ms


In [ ]:
data_split

In [9]:
def create_triples(data_split):
    triples = []
    relations = []
    for n in data_split:
        for li in data_split[n]:
            idx = 0
            for idx in range(len(li)-1):
                tmp_list = [n]
                tmp_list.extend([li[idx],li[idx+1]])
                relations.append(li[idx])
                triples.append(tmp_list)
                idx += 1
    return triples,relations
            

In [10]:
%%time
triples, relations = create_triples(data_split)

Wall time: 150 ms


In [14]:
triples

[['Box set แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกแข็ง)',
  'สำนักพิมพ์ ',
  ' นานมีบุ๊คส์'],
 ['Box set แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกแข็ง)',
  'บาร์โค้ด ',
  ' 3900010020135'],
 ['Box set แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกแข็ง)', 'จำนวนหน้า ', ' 0'],
 ['Box set แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกแข็ง)',
  'ขนาด ',
  ' 23.5 x 27.7 x 4.5 cm'],
 ['Box set แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกแข็ง)', 'น้ำหนัก ', ' 6 kg'],
 ['Box set แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกแข็ง)',
  'หมวดหมู่ ',
  ' นิยาย นิยายแฟนตาซี'],
 ['แพ็กชุด แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกอ่อน)',
  'สำนักพิมพ์ ',
  ' นานมีบุ๊คส์'],
 ['แพ็กชุด แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกอ่อน)',
  'บาร์โค้ด ',
  ' 3900010020142'],
 ['แพ็กชุด แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกอ่อน)', 'จำนวนหน้า ', ' 0'],
 ['แพ็กชุด แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกอ่อน)',
  'ขนาด ',
  ' 23.5 x 27.7 x 4.5 cm'],
 ['แพ็กชุด แฮร์รี่ พอตเตอร์ ฉบับครบรอบ 20ปี (ปกอ่อน)',
  'น้ำหนัก ',
  ' 4.315 kg'],
 ['แพ็กชุด แฮร์รี่ พอตเตอร์ ฉบับครบรอบ

In [11]:
def save_to_txt(triples, file_name):
# Write the triples to text file
    with open('./Data/triples_{}.txt'.format(file_name), 'w', encoding='utf-8') as f:
        for _list in triples:
            for _string in _list:
                #f.seek(0)
                f.write(str(_string) + '\t')
            f.write('\n')
    return

In [12]:
save_to_txt(triples, file_name)